# Grizzly
Grizzly implementiert DataFrames, die möglichst viele Operationen im Datenbankmanagementsystem berechnen, statt zuerst die Daten in den lokalen Arbeitsspeicher zu übertragen. In diesem Abschnitt wird der Unterschied in der Ausführungsgeschwindigkeit zwischen Pandas und Grizzly an Hand eines Beispiels, bei dem viele Tupel aus der Datenbank übertragen werden müssen, beleuchtet.

In [ ]:
import random
import sqlite3
import string
from contextlib import closing

import pandas as pd

import grizzly
from grizzly.relationaldbexecutor import RelationalExecutor
from grizzly.sqlgenerator import SQLGenerator

## Inhaltsverzeichnis
- [Datengenerierung](#Datengenerierung)
- [Join mit Pandas](#Join-mit-Pandas)
- [Join mit SQL](#Join-mit-SQL)
- [Join mit Grizzly](#Join-mit-Grizzly)
- [Evaluierung](#Evaluierung)

## Datengenerierung
Diese Zelle erstellt eine Datenbank mit zwei Tabellen und fügt in jede $1.000.000$ Tupel ein. Die Datenbank wird vollständig im Arbeitsspeicher behalten.

In [ ]:
db = sqlite3.connect(':memory:')
with closing(db.cursor()) as cur:
    for table, id_offset in ('table1', 1), ('table2', 6):
        cur.execute(f'CREATE TABLE IF NOT EXISTS {table} (id INTEGER PRIMARY KEY, name TEXT)')
        for i in range(id_offset, 1_000_000 + id_offset):
            random_name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=32))
            cur.execute(f'INSERT INTO {table} VALUES ({i}, "{random_name}")')

Im Folgenden sollen die Namen der IDs extrahiert werden, die nur in einer der beiden Tabellen vorkommen.

## Join mit Pandas
Zunächst werden beide Tabelle in ein DataFrame geladen. Anschließend können beide Tabellen durch einen Outer-Join verknüpft werden. (Die DataFrames `table1` und `table2` könnten nun gelöscht werden.) Zuletzt lässt sich das Ergebnis filtern.

In [ ]:
%%time

table1 = pd.read_sql_query('SELECT * FROM table1', db, index_col='id')
table2 = pd.read_sql_query('SELECT * FROM table2', db, index_col='id')

result = table1.join(table2, how='outer', lsuffix='l', rsuffix='r')

result[result['namel'].isna() | result['namer'].isna()]

## Join mit SQL
Auf Grund von Beschränkungen bezüglich Outer Joins in SQLite ist das Statement aus zwei Left Joins zusammengesetzt.

In [ ]:
%%time

pd.read_sql_query('''
    SELECT table1.id, table1.name as namel, NULL as namer
    FROM table1
    LEFT JOIN table2
        ON table1.id = table2.id
    WHERE table2.name IS NULL

    UNION

    SELECT table2.id, NULL as namel, table2.name as namer
    FROM table2
    LEFT JOIN table1
        ON table2.id = table1.id
    WHERE table1.name IS NULL
''', db, index_col='id')

## Join mit Grizzly
Grizzly bietet eine Schnittstelle, die Pandas ähnelt. Im Hintergrund wird jedoch auf Basis der Operationen ein SQL-Statement generiert und zur Datenbank gesendet. Das Ziel ist dabei, einen möglichst großen Teil der Berechnungen dem Datenbankmanagementsystem zu überlassen, um dort vorgehaltene Informationen über die Datenstrukturen zu nutzen, während die Größe der zu übertragenden Daten sinkt.

Grizzly implementiert nicht alle Operationen aus Pandas. Weiterhin ist Grizzly natürlich von der verwendeten Datenbank-Software abhängig. Demnach muss bei der Verwendung mit SQLite ebenfalls der Outer Join nachgebildet werden.

In [ ]:
grizzly.use(RelationalExecutor(db, SQLGenerator("sqlite")))

In [ ]:
%%time

# Tabellen lesen und Spalten umbenennen
table1 = grizzly.read_table('table1')
table1['namel'] = table1['name']

table2 = grizzly.read_table('table2')
table2['namer'] = table2['name']

# Joins durchführen
result1 = table1.join(table2, how='left', on=['id', 'id'])
result1 = result1[result1['namer'] == None][['id', 'namel']]

result2 = table2.join(table1, how='left', on=['id', 'id'])
result2 = result2[result2['namel'] == None][['id', 'namer']]

# Ergebnis in "normale" DataFrames kopieren
result1 = pd.DataFrame(dict(result1.items())).set_index('id')
result2 = pd.DataFrame(dict(result2.items())).set_index('id')

# Ergebnisse verknüpfen
result = pd.concat([result1, result2])
result

## Evaluierung
Die Tabelle listet die benötigte Zeit auf, welche die verschiedenen Methoden auf einem betagten Notebook benötigen. Vergleichen Sie diese hinsichtlich der Verhältnisse mit Ihren eigenen Ergebnissen.

| Methode        | Zeit in Millisekunden |
| -------------- | --------------------- |
| Pandas         | 2170                  |
| Join in SQLite | 195                   |
| Grizzly        | 251                   |

Festhalten lässt sich, dass es lohnenswert sein kann, Daten aus Datenbanken nicht komplett in Pandas zu laden und stattdessen einige Operationen dem Datenbankmanagementsystem zu überlassen.